In [23]:
import os
import numpy as np
import pandas as pd
import scipy.stats as sc
import plotly.tools as tls
import plotly.plotly as py
import plotly.graph_objs as go
%run L2M_Functions.py
%matplotlib inline

In [3]:
path = os.getenv('DATAPATH') + '/NBA_L2M_Reports/ReportData2015_2016/'

NBA_cities = pd.DataFrame.from_csv('NBA_cities.csv')
abbr_dict = NBA_cities[['Team', 'Abbr']].set_index('Team').to_dict()['Abbr']

NBA_players_2015 = pd.DataFrame.from_csv('NBA_Players_2015.csv')
NBA_players_2015['Player'] = NBA_players_2015['Player'].apply(lambda x: x.split('''\\''')[0])
NBA_players_2015.set_index('Player', inplace=True)

In [4]:
p_calls = Incorrect_Calls(path, len(os.listdir(path)))

player_calls = pd.DataFrame(p_calls.items(), columns=['Player', 'Calls_Against'])
player_calls.set_index('Player', inplace=True)

% 100.0


In [5]:
reports_2015 = Team_Reports(path, len(os.listdir(path)))

team_reports_2015 = pd.DataFrame(reports_2015.items(), columns=['Tm', 'L2M_Reports'])
team_reports_2015.replace(abbr_dict, inplace=True)
team_reports_2015.replace({'Suns': 'PHO', 'Magic': 'ORL'}, inplace=True)

% 100.0


In [6]:
NBA_player_calls = NBA_players_2015.join(player_calls, how='inner')
NBA_player_calls.to_csv('NBA_2015_playercalls.csv')

In [7]:
team_calls = NBA_player_calls[['Tm', 'Calls_Against']].groupby('Tm').sum()
team_results = team_calls.join(team_reports_2015.set_index('Tm'), how='inner')
team_results['CA_PG'] = team_results['Calls_Against'] / team_results['L2M_Reports']
team_results['ZScore'] = sc.mstats.zscore(team_results.CA_PG)
team_results.sort_values('ZScore', ascending=False)
team_results.reset_index(inplace=True)
team_results.replace(NBA_cities[['Abbr']].reset_index().set_index('Abbr').to_dict()['Location'],
                     inplace=True)
team_results.set_index('Tm', inplace=True)
team_results.to_csv('NBA_2015_teamcalls.csv')

In [8]:
tls.set_credentials_file(username='jvani', api_key=os.getenv('PLOTLY_API'))

r=['Reports: ' + str(L2Ms) for L2Ms in team_results.sort_index().L2M_Reports]
cr=['Calls/Report: ' + str(CA.round(2)) for CA in team_results.sort_index().CA_PG]
z=['Z Score: ' + str(Z.round(2)) for Z in team_results.sort_index().ZScore]

annot = []
for i in range(len(r)):
    t = r[i] + '<br>' + cr[i] + '<br>' + z[i]
    annot.append(t)

NBA_CA = go.Bar(x=team_results.index.sort_values(),
                y=team_results.sort_index().Calls_Against,
                text=annot,
                marker=dict(
        color=['rgb(225, 58, 62)', 'rgb(0, 131, 72)', 'rgb(6, 25, 34)',
               'rgb(29, 17, 96)', 'rgb(206, 17, 65)', 'rgb(134, 0, 56)',
               'rgb(0, 125, 197)', 'rgb(77, 144, 205)', 'rgb(237, 23, 76)',
               'rgb(253, 185, 39)', 'rgb(206, 17, 65)', 'rgb(255, 198, 51)',
               'rgb(237, 23, 76)', 'rgb(253, 185, 39)', 'rgb(15, 88, 108)',
               'rgb(152, 0, 46)', 'rgb(0, 71, 27)', 'rgb(0, 80, 131)',
               'rgb(0, 43, 92)', 'rgb(0, 107, 182)', 'rgb(0, 125, 195)',
               'rgb(0, 125, 197)', 'rgb(237, 23, 76)', 'rgb(229, 96, 32)',
               'rgb(224, 58, 62)', 'rgb(114, 76, 159)', 'rgb(186, 195, 201)',
               'rgb(206, 17, 65)', 'rgb(0, 43, 92)', 'rgb(0, 43, 92)']))

data = [NBA_CA]
layout = go.Layout(
    title='NBA Last 2 Minute Report Calls Against',
    yaxis=dict(title='Calls Against in L2M Reports'),
    margin = dict(b = 110))

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='NBA_L2M_CallsAgainst')

In [25]:
tls.set_credentials_file(username='jvani', api_key=os.getenv('PLOTLY_API'))

c=['Total Calls: ' + str(calls) for calls in team_results.sort_index().Calls_Against]
r=['Reports: ' + str(L2Ms) for L2Ms in team_results.sort_index().L2M_Reports]
z=['Z Score: ' + str(Z.round(2)) for Z in team_results.sort_index().ZScore]

annot = []
for i in range(len(r)):
    t = c[i] + '<br>' + r[i] + '<br>' + z[i]
    annot.append(t)

NBA_CA = go.Bar(x=team_results.index.sort_values(),
                y=team_results.sort_index().CA_PG,
                text=annot,
                marker=dict(
        color=['rgb(225, 58, 62)', 'rgb(0, 131, 72)', 'rgb(6, 25, 34)',
               'rgb(29, 17, 96)', 'rgb(206, 17, 65)', 'rgb(134, 0, 56)',
               'rgb(0, 125, 197)', 'rgb(77, 144, 205)', 'rgb(237, 23, 76)',
               'rgb(253, 185, 39)', 'rgb(206, 17, 65)', 'rgb(255, 198, 51)',
               'rgb(237, 23, 76)', 'rgb(253, 185, 39)', 'rgb(15, 88, 108)',
               'rgb(152, 0, 46)', 'rgb(0, 71, 27)', 'rgb(0, 80, 131)',
               'rgb(0, 43, 92)', 'rgb(0, 107, 182)', 'rgb(0, 125, 195)',
               'rgb(0, 125, 197)', 'rgb(237, 23, 76)', 'rgb(229, 96, 32)',
               'rgb(224, 58, 62)', 'rgb(114, 76, 159)', 'rgb(186, 195, 201)',
               'rgb(206, 17, 65)', 'rgb(0, 43, 92)', 'rgb(0, 43, 92)']))

data = [NBA_CA]
layout = go.Layout(
    title='NBA Last 2 Minute Report Calls Against Per Report',
    yaxis=dict(title='Calls Against Per Report'),
    margin = dict(b = 110))

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='NBA_L2M_CallsAgainstPerReport')